## Einrichtung von Julia JuMP
---

Überprüfen Sie die bereits installierten Pakete. Es sind möglicherweise alle notwendigen Pakete bereits vorhanden.

In [1]:
using Pkg;
Pkg.status()

Status `C:\Users\krueg\.julia\environments\v1.4\Project.toml`
  [9961bab8] Cbc v0.7.0
  [e2554f3b] Clp v0.8.0
  [60bf3e95] GLPK v0.13.0
  [2e9cd046] Gurobi v0.8.0
  [7073ff75] IJulia v1.21.2
  [4076af6c] JuMP v0.21.2
  [f3b207a7] StatsPlots v0.14.6


Installieren Sie ``JuMP`` , mit dem Sie einfach Optimierungsprogramme formulieren können, sowie ``Cbc``, einen open-source Solver zum Lösen des Problems, und ``StatsPlots`` zur Visualisierung der Lösung.



In [2]:
Pkg.add("JuMP")
Pkg.add("Cbc");
Pkg.add("StatsPlots");

   Updating registry at `C:\Users\krueg\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `C:\Users\krueg\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\krueg\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\krueg\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\krueg\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\krueg\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\krueg\.julia\environments\v1.4\Manifest.toml`
 [no changes]


Initialisieren Sie die installierten Pakete, damit sie im folgenden Notebook verwendet werden können.


In [3]:
using JuMP, Cbc, StatsPlots;

# Aggregierte Planung

### Entscheidungsvariablen:

Bestimmung der optimalen **Produktionsmengen, Absatzmengen, Fremdbezugsmengen** und **Lagerbestände** der Produkte, sowie der optimal zu nutzenden **Zusatzkapazität** der Ressourcen je Periode, 

### Zielfunktion:

0) **Deckungsbeitragsmaximierung**: $$  \qquad \max db = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (e_iA_{it}-k_i^vX_{it}-k_i^l L_{it}-k_i^fF_{it}) - \sum_{j=1}^J \sum_{t=1}^T (k_j^zZ_{jt}) $$


### Nebenbedingungen:

1) **Kapazitaetsrestriktion:** $ \hspace{41mm} \sum_{i=1}^I(r_{ij}\cdot X_{it}) \leq c_j + z_{jt} \hspace{40mm} \forall j \in J, t \in T $

2) **Absatzobergrenze:** $\hspace{38mm} \qquad A_{it} \leq d_{it} \hspace{69mm} \forall i \in I , t \in T $


3) **Absatzuntergrenze:** $ \hspace{37mm}\qquad A_{it} \geq d_{it}^{min} \hspace{66mm} \forall i \in I , t \in T $
 
4) **Lagerbilanz:** $ \hspace{61mm} L_{it}=L_{i,t-1}+X_{it}-A_{it}+F_{it} \hspace{31mm} \forall i \in I , t \in T$

5) **Anfangslagerbestand:**   $ \hspace{43mm} L_{i,0} = l_{start} \hspace{64mm} \forall  i \in I$


6) **Max. Zusatzkapazität:** $ \hspace{44mm} Z_{jt} \leq z_j^{max}  \hspace{65mm} \forall j \in J, t \in T $

7) **Max. Fremdbezug:** $ \hspace{49mm} F_{it} \leq f_i^{max} \hspace{65mm} \forall i \in I, t \in T $

8) **Nichtnegativitätsbedingung:** $ \hspace{31mm} X_{it}, A_{it}, L_{it}, F_{it} \geq 0 \hspace{49mm} \forall i \in I, t \in T$

und $\hspace{81mm} Z_{jt} \geq 0 \hspace{71mm}  \forall j \in J, t \in T $ 

* * *

## Verwendete Symbole

### Mengen

$i \in (1,..,I)\hspace{20mm} $ Produkte  

$j \in (1,..,J)\hspace{20mm}$  Ressourcen 

$t \in (1,..,T)\hspace{20mm}$  Zeiten 

### Variablen

$X_{it}$  $\geq0$      $\hspace{28mm}$ Produktionsmenge      

$A_{it}$  $ \geq 0 $  $\hspace{28mm} $ Absatzmenge 

$L_{it}$  $\geq0$  $\hspace{28mm} $ Lagerbestand  

$F_{it}$       $ \geq0$    $\hspace{28mm} $ Fremdbezug   

$Z_{jt}$      $\geq0$      $\hspace{27mm} $ Zusatzkapazität  


### Parameter

$e_i \hspace{39mm}$   Erlös    

$k_i^v \hspace{38mm}$ variable Kosten

$k_i^l \hspace{38mm}$ Lagerkostensatz 

$k_j^z \hspace{38mm}$ Überstundenkostensatz 

$k_i^f \hspace{38mm}$ Fremdbezugskosten 

$d_{it} \hspace{38mm}$  Nachfrage  

$d_{it}^{min} \hspace{36mm}$ Absatzuntergrenze  

$r_{ij} \hspace{38mm}$  Produktionskoeffizient 

$c_j \hspace{39mm}$  Kapazität  

$l_{start} \hspace{34mm}$  Anfangslagerbestand  

$z_j^{max} \hspace{34mm}$ max. Zusatzkapazität 

$f_i^{max} \hspace{33mm}$  max. Fremdbezug

 * * *
 

## Erstellen Sie das Modell namens ``m`` und geben Sie als zu verwendenden Solver Cbc an.
---

In [4]:
m = Model(Cbc.Optimizer);

### Mengen und Parameter
---

Fügen Sie die Mengen ein.

In [5]:
#Mengen
Produkte = ["Damen", "Herren", "Kinder"]  
Ressourcen = ["Herstellen"]
Perioden = ["1", "2", "3", "4", "5", "6"]

#Längen
I = length(Produkte);
J = length(Ressourcen);
T = length(Perioden);

Fügen Sie die Parameter ein.

In [6]:
r = [6.3/60, 5.7/60, 3.9/60];
#Ressourcenverbrauch von Ressource j durch Produkt i
#Hinweis: Eineheiten beachten (Stunden/Minuten)

       
d = [  1600 1400 1000 1200 1600 1400     
       1000 1200 800  1000 1600 2000    
       1200 1000 800  1400 1000 1400 ];
#Nachfrage von Produkt i in Periode t
 
        
dmin = d;
#Absatzuntergrenze von Produkt i in Periode t
#Hinweis: Die Nachfrage muss komplett bedient werden

In [7]:
c = [320];                           #Kapazität der Ressource j
#Hinweis: gehen Sie vereinfachend davon aus, ein Monat hat vier Wochen

e = [300, 400, 200];                 #Erlös des Produktes

kf =   [240, 324, 180];              #Fremdbezugskostensatz 2 der Produkte (Nur langfr. Fremdbezug: günstigeren Kostensatz verwenden)

kl =  [3.2, 4.32, 2.4];              #Lagerkostensatz der Produkte

kz =   [450];                        #Überstundenkostensatz von Ressource j

kv =  [160, 216, 120];               #variable Herstellkosten der Produkte

zmax =  [40];                        #Max. Zusatzkapazität von Ressource j

l_start =   [0, 0, 0];               #Anfangslagerbestand der Produkte


           
fmax = [   2000 2000 2000 2000 2000 2000          
           2000 2000 2000 2000 2000 2000    
           2000 2000 2000 2000 2000 2000]
#Maximaler Fremdbezug von Produkt i in Periode t

3×6 Array{Int64,2}:
 2000  2000  2000  2000  2000  2000
 2000  2000  2000  2000  2000  2000
 2000  2000  2000  2000  2000  2000

### Entscheidungsvariablen
---
Definieren Sie die Entscheidungsvariablen. Achten Sie auf die Definitionsbereiche:

 **Nichtnegativitätsbedingung**: Die Entscheidungsvariablen, also die Produktions- und Absatzmengen, Lagerbestände, Fremdbezugsmengen und die in Anspruch genommenen Zusatzkapazitäten dürfen nur positive Werte annehmen, wir setzen deren Nichtnegativität voraus.

$ \qquad X_{it}, A_{it}, L_{it}, F_{it} \geq 0 \qquad \qquad \forall i \in I, t \in T$

und $\qquad \qquad Z_{jt} \geq 0 \qquad \qquad \qquad  \forall j \in J, t \in T $ 

In [8]:
@variables  m begin
            X[1:I,1:T]>=0,Int  #Produktionsmenge von Produkt i in Periode t
            A[1:I,1:T]>=0,Int  #Absatzmenge von Produkt i in Periode t
            F[1:I,1:T]>=0,Int  #Fremdbezugsmenge von Produkt i in Periode t
            Z[1:J,1:T]>=0,Int  #Zusatzkapazität von Ressource j in Periode t
            L[1:I,0:T]>=0,Int; #Lagerbestand von Produkt i am Periodenende von t
            end

## Zielfunktion

 **Deckungsbeitragsmaximierung:** Der Gesamtdeckungsbeitrag db soll maximiert werden. Dieser berechnet sich hier aus der Summe der einzelnen Produkterlöse abzüglich der variablen Kosten, abzüglich der Lagerkosten, abzüglich der Fremdbezugskosten und wird Zusätzlich noch verringert um die Zusatzkapazitätskosten der Ressourcen.

$  \qquad \max db = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (e_iA_{it}-k_i^vX_{it}-k_i^l L_{it}-k_i^fF_{it}) - \sum_{j=1}^J \sum_{t=1}^T (k_j^zZ_{jt}) $

In [9]:
@objective(m, Max,  sum(e[i] * A[i,t] - kv[i] * X[i,t] - kl[i] * L[i,t] - kf[i] * F[i,t] 
           for i=1:I for t=1:T) - sum(kz[j] *Z[j,t] for j=1:J for t=1:T));

## Nebenbedingungen

**Kapazitätsrestriktion**: Die zur Verfügung stehende Kapazität für die auf den einzelnen Ressourcen mögliche Produktionszeit darf nicht überschritten werden. Die Summe der Gesamtproduktionszeit aller Produkte $ i = 1,...,I $ auf den einzelnen Ressourcen j darf in jeder Periode die jeweils verfügbare Kapazität  $𝑐_j$ ergänzt um die Zusatzkapazität $o_{jt}$  nicht überschreiten.

$ \qquad \sum_{i=1}^I(r_{ij}\cdot X_{it}) \leq c_j + z_{jt} \qquad \forall j \in J, t \in T $

In [10]:
@constraint(m, KapRes[t=1:T,j=1:J], sum(r[i,j] * X[i,t] for i=1:I) <= c[j] + Z[j,t]);

 **Absatzuntergrenze**: Für die Produkte existieren Absatzuntergrenzen basierend auf der Nachfrage. Die abgesetzte Menge der einzelnen Produkte kann diese Absatzuntergrenze in keiner Periode unterschreiten.

$ \qquad A_{it} \geq d_{it}^{min} \qquad \qquad \qquad \qquad \forall i \in I , t \in T $

In [11]:
@constraint(m, AbsUn[i=1:I,t=1:T], A[i,t] >= dmin[i,t]);

 **Absatzobergrenze**: Für die Produkte existieren Absatzober- und untergrenzen basierend auf der Nachfrage. Die abgesetzte Menge der einzelnen Produkte kann diese Absatzobergrenze in keiner Periode überschreiten.

$ \qquad A_{it} \leq d_{it} \qquad \qquad \qquad \qquad \forall i \in I , t \in T $

In [12]:
@constraint(m, AbsOb[i=1:I,t=1:T], A[i,t] <= d[i,t]);

 **Lagerbilanz**: Der Lagerbestand eines Produktes am Ende einer Periode berechnet sich aus der Summe der eingelagerten Menge in der Vorperiode, der Produktionsmenge und der Fremdbezugsmenge, abzüglich der abgesetzen Menge des Produktes.

$ \qquad L_{it}=L_{i,t-1}+X_{it}-A_{it}+F_{it} \qquad \forall i \in I , t \in T$

In [13]:
@constraint(m, Lager[i=1:I,t=1:T], L[i,t] == L[i,t-1] + X[i,t] + F[i,t] - A[i,t]);

**Anfangslagerbestand**: Der Anfangslagerbestand aller Produkte entspricht dem initial gesetzen $l_i$.
 
  $$ \qquad L_{i,0} = l_{start} \hspace{40mm} \forall  i \in I$$


In [14]:
@constraint(m, AnfLager[i=1:I], L[i,0] == l_start[i]);

 **Max. Zusatzkapazität**: Die eingesetzten Zusatzkapazitäten (Überstunden) für jede Ressource müssen in jeder Periode unter der Obergrenze, der maximalen Zusatzkapazität, bleiben. Die maximale Zusatzkapazität ist für jede Ressource j einzeln gegeben.

$ \qquad Z_{jt} \leq z_j^{max}  \qquad \qquad \qquad \qquad \forall j \in J, t \in T $

In [15]:
@constraint(m, ZusatzKapa[j=1:J,t=1:T], Z[j,t] <= zmax[j] );

 **Max. Fremdbezug**: Der dazugekaufte Fremdbezug jedes Produktes muss in allen Perioden unter der Obergrenze, dem maximalen Fremdbezug, bleiben. Der maximale Fremdbezug ist für jedes Produkt i einzeln gegeben. 

$ \qquad F_{it} \leq f_i^{max} \qquad \qquad \qquad \qquad \forall i \in I, t \in T $

In [16]:
@constraint(m, MaxFB[i=1:I,t=1:T], F[i,t] <= fmax[i,t] );

## Lösen Sie das Modell.
---

In [17]:
optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3.03505e+06 - 0.02 seconds
Cgl0003I 0 fixed, 13 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 24 rows, 60 columns (60 integer (0 of which binary)) and 93 elements
Cbc0012I Integer solution of -3034609 found by DiveCoefficient after 0 iterations and 0 nodes (0.07 seconds)
Cbc0038I Full problem 24 rows 60 columns, reduced to 3 rows 6 columns
Cbc0012I Integer solution of -3034610.2 found by DiveCoefficient after 234 iterations and 0 nodes (0.19 seconds)
Cbc0031I 4 added rows had average density of 35
Cbc0013I At root node, 4 cuts changed objective from -3035054.8 to -3035051.7 in 91 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 3 column cuts (3 active)  in 0.014 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 264 row cuts average 30.3

Lassen Sie sich das Modell anzeigen.

In [18]:
m

A JuMP Model
Maximization problem with:
Variables: 81
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 21 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 18 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 48 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 81 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 81 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: A, AbsOb, AbsUn, AnfLager, F, KapRes, L, Lager, MaxFB, X, Z, ZusatzKapa

Lassen Sie sich den Zielfunktionswert anzeigen.

In [19]:
db = JuMP.objective_value(m)

println("Objective value db: ", round(Int64,db))

Objective value db: 3035050


Platz für weitere Berechnungen:

In [20]:
JuMP.value.(Z)

1×6 Array{Float64,2}:
 21.0  6.0  0.0  0.0  38.0  40.0

In [21]:
JuMP.value.(F)

3×6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [22]:
JuMP.value.(A)

3×6 Array{Float64,2}:
 1600.0  1400.0  1000.0  1200.0  1600.0  1400.0
 1000.0  1200.0   800.0  1000.0  1600.0  2000.0
 1200.0  1000.0   800.0  1400.0  1000.0  1400.0

In [23]:
JuMP.value.(X)

3×6 Array{Float64,2}:
 1600.0  1400.0  1823.0  1279.0  1345.0   753.0
 1000.0  1200.0   800.0  1001.0  1599.0  2000.0
 1200.0  1000.0   809.0  1394.0   998.0  1399.0

In [24]:
JuMP.value.(L)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, 0:6
And data, a 3×7 Array{Float64,2}:
 0.0  0.0  0.0  823.0  902.0  647.0  0.0
 0.0  0.0  0.0    0.0    1.0    0.0  0.0
 0.0  0.0  0.0    9.0    3.0    1.0  0.0

In [25]:
round(Int64, sum(JuMP.value.(Z[j,t])*kz for t in 1:T for j in 1:J))


MethodError: MethodError: no method matching round(::Type{Int64}, ::Array{Float64,1})
Closest candidates are:
  round(::Type{T}, !Matched::Integer) where T<:Integer at int.jl:531
  round(::Type{T}, !Matched::Float16) where T<:Integer at float.jl:368
  round(::Type{T}, !Matched::BigFloat) where T<:Union{Signed, Unsigned} at mpfr.jl:314
  ...

In [26]:
#Kosten für Überstunden
sum(JuMP.value.(Z[j,t])*kz for t in 1:T for j in 1:J)

1-element Array{Float64,1}:
 47250.0

In [27]:
#Erlöse Damen
round(Int64, sum(JuMP.value.(A[1,t])*e[1] for t in 1:T ))

2460000

In [28]:
#Erlöse Herren
round(Int64, sum(JuMP.value.(A[2,t])*e[2] for t in 1:T))

3040000

In [29]:
#Erlöse Kinder
round(Int64, sum(JuMP.value.(A[3,t])*e[3] for t in 1:T))

1360000

In [30]:
#Kosten Produktion Damen
round(Int64, sum(JuMP.value.(X[1,t])*kv[1] for t in 1:T))

1312000

In [31]:
#Kosten Produktion Herren
round(Int64, sum(JuMP.value.(X[2,t])*kv[2] for t in 1:T))

1641600

In [32]:
#Kosten Produktion Kinder
round(Int64, sum(JuMP.value.(X[3,t])*kv[3] for t in 1:T))

816000

In [33]:
#Kosten Lagerung Damen
round(Int64, sum(JuMP.value.(L[1,t])*kl[1] for t in 1:T))

8065

In [34]:
#Kosten Lagerung Herren
round(Int64, sum(JuMP.value.(L[2,t])*kl[2] for t in 1:T))

4

In [35]:
#Kosten Lagerung Kinder
round(Int64, sum(JuMP.value.(L[3,t])*kl[3] for t in 1:T))

31

In [36]:
#Kosten Produktion gesamt
round(Int64, sum(JuMP.value.(X[i,t])*kv[i] for t in 1:T for i in 1:I))

3769600

In [37]:
#Menge Produktion gesamt
round(Int64, sum(JuMP.value.(X[i,t]) for t in 1:T for i in 1:I))

22600

In [38]:
#Erlöse gesamt
round(Int64,sum(JuMP.value.(A[i,t])*e[i] for t in 1:T for i in 1:I))

6860000

In [39]:
sum(JuMP.value.(L[3,1])*kl[3])

0.0